In [2]:
import pandas as pd
import numpy as np

from keras.preprocessing import sequence

# train data
training_data_path = '../../datasets/train_data/series/'
result_path = '../../datasets/results/'
def series_features(series,maxlen=20):
    sec1 = series.apply(lambda x :[int(float(i)*100000) for i in x.split(' ')]).tolist()
    return sequence.pad_sequences(sec1, maxlen=maxlen) + 40000

series_names = ['unbiased','hourly_corrected','daily_corrected','global_corrected']
maxlen = 20

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Dense, Embedding,Flatten
from keras.models import Sequential
from keras.layers import LSTM
from keras.optimizers import Adam

def LSTM_regression(X,y,val_rate=0.3):
    LR = 0.01
    length_val = int(len(y)*0.3)

    X_train, y_train = X[length_val:,:],y[length_val:]
    X_test, y_test =X[:length_val,:],y[:length_val]

    X_train = np.reshape(X_train, (X_train.shape[0],1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))
    print(X_train.shape)
    # build RNN model
    model = Sequential()
    model.add(LSTM(input_dim=20,
                   output_dim=50,return_sequences=True))
    model.add(Flatten())
    model.add(Dense(output_dim=1))
    # optimizer
    model.compile(optimizer='rmsprop',
                  loss='mean_squared_error')

    # training
    model.fit(X_train, y_train,nb_epoch=3,validation_data=(X_test, y_test))
    score = model.evaluate(X_test, y_test)

    print('test cost: ', score)
    return score





def LSTM_classifer(X,y,val_rate=0.3):
    LR = 0.01
    length_val = int(len(y)*0.3)
    num_classes = len(set(y))

    X_train, y_train = X[length_val:,:],y[length_val:]
    X_test, y_test =X[:length_val,:],y[:length_val]
    
    # data pre-processing
    y_train = np_utils.to_categorical(y_train, num_classes=num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes=num_classes)

    # build RNN model
    model = Sequential()

    # RNN cell


    model.add(Embedding(200000, 128))

    model.add(LSTM(128, dropout=0.6, recurrent_dropout=0.2))
    # model.add(LSTM(128))
    model.add(Dense(num_classes, activation='softmax'))

    # optimizer
    model.compile(optimizer=Adam(LR),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # training
    model.fit(X_train, y_train,nb_epoch=3,validation_data=(X_test, y_test))
    score, acc = model.evaluate(X_test, y_test)

    print('test cost: ', score, 'test accuracy: ', acc)
    return acc

In [8]:
game_name = 'soccer'
df = pd.DataFrame()
dic = {}
for series_name in series_names:
    series_df = pd.read_csv(training_data_path+game_name+'_'+series_name+'.csv')
    sec1 = series_features(series_df['sec1'],maxlen=maxlen)
    sec2 = series_features(series_df['sec2'],maxlen=maxlen)
    sec3 = series_features(series_df['sec3'],maxlen=maxlen)

    dic['sec1_home_win'] = LSTM_classifer(sec1,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec2_home_win'] = LSTM_classifer(sec2,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec3_home_win'] = LSTM_classifer(sec3,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec1_direction'] = LSTM_classifer(sec1,series_df['direction_2'].apply(int).values,val_rate=0.3)
    dic['sec2_direction'] = LSTM_classifer(sec2,series_df['direction_3'].apply(int).values,val_rate=0.3)
    pred = pd.Series(dic) 
    pred.name = series_name
    df = df.append(pred)
df.to_csv(result_path+'soccer_rnn.csv')

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:71: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2693 samples, validate on 1153 samples
Epoch 1/3
2693/2693 [==============================] - 61s 23ms/step - loss: 0.6972 - acc: 0.5195 - val_loss: 0.6751 - val_acc: 0.5690
Epoch 2/3
2693/2693 [==============================] - 65s 24ms/step - loss: 0.6686 - acc: 0.5941 - val_loss: 0.6614 - val_acc: 0.6331
Epoch 3/3
1153/1153 [==============================] - 1s 800us/step
test cost:  0.6729930563404161 test accuracy:  0.6027753686036427
Train on 2693 samples, validate on 1153 samples
Epoch 1/3
2693/2693 [==============================] - 102s 38ms/step - loss: 0.6940 - acc: 0.5462 - val_loss: 0.6675 - val_acc: 0.5993
Epoch 2/3
2693/2693 [==============================] - 90s 34ms/step - loss: 0.6529 - acc: 0.6257 - val_loss: 0.6727 - val_acc: 0.6045
Epoch 3/3
1153/1153 [==============================] - 1s 590us/step
test cost:  0.6736141476079059 test accuracy:  0.6209887250650477
Train on 2693 samples, validate on 1153 samples
Epoch 1/3
2693/2693 [========================

In [116]:
game_name = 'iceball'
df = pd.DataFrame()
dic = {}
for series_name in series_names:
    series_df = pd.read_csv(training_data_path+game_name+'_'+series_name+'.csv')
    sec1 = series_features(series_df['sec1'],maxlen=maxlen)
    sec2 = series_features(series_df['sec2'],maxlen=maxlen)
    sec3 = series_features(series_df['sec3'],maxlen=maxlen)
    
    dic['sec1_home_win'] = LSTM_classifer(sec1,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec2_home_win'] = LSTM_classifer(sec2,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec3_home_win'] = LSTM_classifer(sec3,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec1_direction'] = LSTM_classifer(sec1,series_df['direction_2'].apply(int).values,val_rate=0.3)
    dic['sec2_direction'] = LSTM_classifer(sec2,series_df['direction_3'].apply(int).values,val_rate=0.3)
    pred = pd.Series(dic) 
    pred.name = series_name
    df = df.append(pred)
df.to_csv(result_path+'iceball_rnn.csv')

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:70: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 4513 samples, validate on 1934 samples
Epoch 1/3
4513/4513 [==============================] - 182s 40ms/step - loss: 0.6897 - acc: 0.5442 - val_loss: 0.6849 - val_acc: 0.5620
Epoch 2/3
4513/4513 [==============================] - 163s 36ms/step - loss: 0.6773 - acc: 0.5894 - val_loss: 0.7213 - val_acc: 0.5295
Epoch 3/3
1934/1934 [==============================] - 9s 5ms/step
test cost:  0.7183501133011441 test accuracy:  0.520165460124504
Train on 4513 samples, validate on 1934 samples
Epoch 1/3
4513/4513 [==============================] - 188s 42ms/step - loss: 0.6929 - acc: 0.5446 - val_loss: 0.6994 - val_acc: 0.5129
Epoch 2/3
4513/4513 [==============================] - 164s 36ms/step - loss: 0.6763 - acc: 0.5766 - val_loss: 0.6906 - val_acc: 0.5584
Epoch 3/3
1934/1934 [==============================] - 9s 5ms/step
test cost:  0.7177997690691943 test accuracy:  0.5351602895553258
Train on 4513 samples, validate on 1934 samples
Epoch 1/3
4513/4513 [==========================

In [9]:
game_name = 'baseball'
df = pd.DataFrame()
dic = {}
for series_name in series_names:
    series_df = pd.read_csv(training_data_path+game_name+'_'+series_name+'.csv')
    sec1 = series_features(series_df['sec1'],maxlen=maxlen)
    sec2 = series_features(series_df['sec2'],maxlen=maxlen)
    sec3 = series_features(series_df['sec3'],maxlen=maxlen)
    
    dic['sec1_home_win'] = LSTM_classifer(sec1,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec2_home_win'] = LSTM_classifer(sec2,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec3_home_win'] = LSTM_classifer(sec3,series_df['home'].apply(int).values,val_rate=0.3)
    dic['sec1_direction'] = LSTM_classifer(sec1,series_df['direction_2'].apply(int).values,val_rate=0.3)
    dic['sec2_direction'] = LSTM_classifer(sec2,series_df['direction_3'].apply(int).values,val_rate=0.3)
    pred = pd.Series(dic) 
    pred.name = series_name
    df = df.append(pred)
df.to_csv(result_path+'baseball_rnn.csv')

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:71: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7844 samples, validate on 3361 samples
Epoch 1/3
7844/7844 [==============================] - 212s 27ms/step - loss: 0.6931 - acc: 0.5345 - val_loss: 0.6848 - val_acc: 0.5620
Epoch 2/3
7844/7844 [==============================] - 238s 30ms/step - loss: 0.6861 - acc: 0.5560 - val_loss: 0.7033 - val_acc: 0.5263
Epoch 3/3
3361/3361 [==============================] - 4s 1ms/step
test cost:  0.6949673214685133 test accuracy:  0.5400178518298125
Train on 7844 samples, validate on 3361 samples
Epoch 1/3
7844/7844 [==============================] - 259s 33ms/step - loss: 0.6949 - acc: 0.5269 - val_loss: 0.6873 - val_acc: 0.5594
Epoch 2/3
7844/7844 [==============================] - 283s 36ms/step - loss: 0.6894 - acc: 0.5473 - val_loss: 0.6877 - val_acc: 0.5552
Epoch 3/3
3361/3361 [==============================] - 5s 1ms/step
test cost:  0.6895238728758765 test accuracy:  0.5367450163641774
Train on 7844 samples, validate on 3361 samples
Epoch 1/3
7844/7844 [=========================